<a href="https://colab.research.google.com/github/caiobaptistaa/Extract_ClimateData_ECMWF/blob/main/NCtoShape_Passo_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2> Unindo netCDF4 com Shape e criando um CSV <h2>

In [ ]:
pip install pandas fiona shapely pyproj rtree geopandas pygeos

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import netCDF4 as nc4
import numpy as np
from osgeo import gdal,osr,ogr
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import xarray as xr
import pygeos
import rtree

/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#importando o shapefile
shp = gpd.read_file("/content/drive/MyDrive/Climate_Data_Files/BR_Municipios_2021.shp")
#importando o netCDF4 extraído do ECMWF
nc = nc4.Dataset('/content/drive/MyDrive/Artigo-Clima-Produtiv/precipitation.nc', mode='r') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset = xr.open_dataset(xr.backends.NetCDF4DataStore(nc))
df = dataset.to_dataframe()

df = df.reset_index()

In [ ]:
df.head()

,longitude,latitude,time,tp
0,-74.0,5.0,1980-01-01,0.001495
1,-74.0,5.0,1980-02-01,0.008001
2,-74.0,5.0,1980-03-01,0.002584
3,-74.0,5.0,1980-04-01,0.004015
4,-74.0,5.0,1980-05-01,0.002290


In [ ]:
# use geopandas points_from_xy() para transformar Longitude e Latitude em uma lista de objetos shapely.Point e defini-los como uma geometria enquanto cria o GeoDataFrame
df_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

In [ ]:
print(df_gdf.head())

   longitude  latitude       time        tp                   geometry
0      -74.0       5.0 1980-01-01  0.001495  POINT (-74.00000 5.00000)
1      -74.0       5.0 1980-02-01  0.008001  POINT (-74.00000 5.00000)
2      -74.0       5.0 1980-03-01  0.002584  POINT (-74.00000 5.00000)
3      -74.0       5.0 1980-04-01  0.004015  POINT (-74.00000 5.00000)
4      -74.0       5.0 1980-05-01  0.002290  POINT (-74.00000 5.00000)


In [ ]:
# check CRS coordenadas
shp.crs #shapefile
df_gdf.crs #geodataframe netcdf

In [ ]:
# defina as coordenadas uma iguais as outras
# PointsGeodataframe.crs = PolygonsGeodataframe.crs
df_gdf.crs = shp.crs

In [ ]:
# check coordenadas depois defini-las umas iguais as outras
df_gdf.crs #geodataframe netcdf

<Geographic 2D CRS: EPSG:4674>
Name: SIRGAS 2000
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: Latin America - Central America and South America - onshore and offshore. Brazil - onshore and offshore.
- bounds: (-122.19, -59.87, -25.28, 32.72)
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [ ]:
# Recorta pontos, linhas ou geometrias de polígonos na extensão da máscara.
mask = gpd.clip(df_gdf, shp)

In [ ]:
mask.head()

,longitude,latitude,time,tp,geometry
20707380,-63.0,-12.8,1987-01-01,0.005997,POINT (-63.00000 -12.80000)
20707301,-63.0,-12.8,1980-06-01,0.000991,POINT (-63.00000 -12.80000)
20707296,-63.0,-12.8,1980-01-01,0.009245,POINT (-63.00000 -12.80000)
20707297,-63.0,-12.8,1980-02-01,0.010215,POINT (-63.00000 -12.80000)
20707298,-63.0,-12.8,1980-03-01,0.008402,POINT (-63.00000 -12.80000)


In [ ]:
mask.shape

(34942824, 5)

In [ ]:
mask.to_csv("/content/drive/MyDrive/Artigo-Clima-Produtiv/precipitation.csv", index = False)